<a href="https://colab.research.google.com/github/Gab2697/Human-Activity-Recognition/blob/master/Baseline_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
#import
import pandas as pd
import tensorflow as tf
import numpy as np
import os

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Prepare the data

Function to load data, organize it by surface and add a specifique label depending on the name of the excel file

In [9]:
def load_data(path,keyword,col):

  Window=1926*2 
  df_asphalt = pd.DataFrame()
  df_grass = pd.DataFrame()
  for fname in os.listdir(path):
    if keyword in fname: 
      os.chdir(path)
      pd_xl_file = pd.ExcelFile(fname)
      df = pd_xl_file.parse("Sheet1") 
      dimensions = df.shape
      Middle=int(dimensions[0]/2)
      Participant=pd.read_excel(fname,sheet_name='Sheet1',  usecols = col)
      Participant['label'] = pd.DataFrame(np.ones(dimensions[0]),dtype=int)
      Participant=Participant.iloc[Middle-Window:Middle+Window]
      df_asphalt = df_asphalt.append(Participant)
      

    else:
      os.chdir(path)
      pd_xl_file_grass = pd.ExcelFile(fname)
      df2 = pd_xl_file_grass.parse("Sheet1") 
      dimensions_grass = df2.shape
      Middle_grass=int(dimensions_grass[0]/2)
      Participant_grass=pd.read_excel(fname,sheet_name='Sheet1', usecols = col)
      Participant_grass['label'] = pd.DataFrame(np.zeros(dimensions_grass[0]),dtype=int)
      Participant_grass=Participant_grass.iloc[Middle_grass-Window:Middle_grass+Window]
      df_grass = df_grass.append(Participant_grass)
      
  return df_asphalt, df_grass

Call load function and add specific columns (participant and trial number)

In [ ]:
#P01
path1= '/content/drive/MyDrive/Gab/P01/exported-1/'
df_asphalt_1, df_grass_1= load_data(path1,'sidewalk',"A,B,J,R,Z")
df_asphalt_1['participant_number']=1#add column for participant number
df_grass_1['participant_number']=1#add column for participant number
df_asphalt_1['trial_number']=list(range(1,len(df_asphalt_1)+1))#add column for trial number
df_grass_1['trial_number']=list(range(1,len(df_grass_1)+1))#add column for trial number
#P03
path3= '/content/drive/MyDrive/Gab/P03/exported-emg/'
df_asphalt_3, df_grass_3= load_data(path3,'asphalt',"A,B,D,L,T")
df_asphalt_3['participant_number']=3
df_grass_3['participant_number']=3
df_asphalt_3['trial_number']=list(range(1,len(df_asphalt_3)+1))
df_grass_3['trial_number']=list(range(1,len(df_grass_3)+1))
#P05
path5= '/content/drive/MyDrive/Gab/P05/exported-emg-5/'
df_asphalt_5, df_grass_5= load_data(path5,'asphalt',"A,B,D,L,T")
df_asphalt_5['participant_number']=5
df_grass_5['participant_number']=5
df_asphalt_5['trial_number']=list(range(1,len(df_asphalt_5)+1))
df_grass_5['trial_number']=list(range(1,len(df_grass_5)+1))
#P07
path7= '/content/drive/MyDrive/Gab/P07/exported-emg-7/'
df_asphalt_7, df_grass_7= load_data(path7,'asphalt',"A,B,D,L,T")
df_asphalt_7['participant_number']=7
df_grass_7['participant_number']=7
df_asphalt_7['trial_number']=list(range(1,len(df_asphalt_7)+1))
df_grass_7['trial_number']=list(range(1,len(df_grass_7)+1))
#P09
path9= '/content/drive/MyDrive/Gab/P09/exported-emg-9/'
df_asphalt_9, df_grass_9= load_data(path9,'asphalt',"A,B,D,L,T")
df_asphalt_9['participant_number']=9
df_grass_9['participant_number']=9
df_asphalt_9['trial_number']=list(range(1,len(df_asphalt_9)+1))
df_grass_9['trial_number']=list(range(1,len(df_grass_9)+1))

In [31]:
df_asphalt_7

,X [s],Trigno sensor 1-TA: EMG 1 [Volts],Trigno sensor 2-Gastroc: EMG 2 [Volts],Trigno sensor 3-RF: EMG 3 [Volts],Trigno sensor 4-BF: EMG 4 [Volts],label,participant_number,trial_number
5287,2.745173,1.510643e-06,0.000132,0.000039,-8.509956e-05,1,7,1
5288,2.745692,-7.721065e-06,0.000148,0.000046,-8.778516e-05,1,7,2
5289,2.746212,-9.231709e-06,0.000098,0.000047,-9.147784e-05,1,7,3
5290,2.746731,-1.174945e-05,0.000039,0.000043,-8.929580e-05,1,7,4
5291,2.747250,-1.443504e-05,0.000050,0.000042,-8.358892e-05,1,7,5
...,...,...,...,...,...,...,...,...
12024,6.243231,1.342794e-06,-0.000440,0.000003,2.182040e-06,1,7,624020
12025,6.243750,8.392462e-07,-0.000434,0.000003,6.713969e-07,1,7,624021
12026,6.244269,-3.860533e-06,-0.000422,0.000005,-6.713969e-07,1,7,624022
12027,6.244788,-8.392462e-06,-0.000409,0.000006,0.000000e+00,1,7,624023


Creating a tensor with specific shape (trialxchannelxframe)

In [47]:
def create_tensor(df_a,df_g):
  #tensor asphalt
  trials_a=len(df_a)/(1926*4) # how many trials
  T_asphalt=np.stack(np.split(df_a.values, trials_a), axis=1)
  #tensor grass
  trials_g=len(df_g)/(1926*4) # how many trials
  T_grass=np.stack(np.split(df_g.values, trials_g), axis=1)
  #tensor both conditions
  a=tf.constant(T_asphalt)
  g=tf.constant(T_grass)
  T = tf.concat([a, g], axis=0)

  return T

In [56]:
# T1=create_tensor(df_asphalt_1,df_grass_1)
# print("T1 shape:", T1.shape)

T3=create_tensor(df_asphalt_3,df_grass_3)
print("T3 shape:", T3.shape)

T5=create_tensor(df_asphalt_5,df_grass_5)
print("T5 shape:", T5.shape)

# T7=create_tensor(df_asphalt_7,df_grass_7)
# print("T7 shape:", T7.shape)

# T9=create_tensor(df_asphalt_9,df_grass_9)
# print("T9 shape:", T9.shape)


T3 shape: (15408, 80, 8)
T5 shape: (15408, 80, 8)


Save in drive (pickle)

In [57]:
#only do once
import pickle
def save_pickle(path_pickle,T):
  f = open(path_pickle, 'wb')
  pickle.dump(T, f)
  f.close()

  return f 

#call function
# f1=save_pickle('/content/drive/MyDrive/Gab/Variables_saved/T1.pickle',T1)
f3=save_pickle('/content/drive/MyDrive/Gab/Variables_saved/T3.pickle',T3)
f5=save_pickle('/content/drive/MyDrive/Gab/Variables_saved/T5.pickle',T5)
# f7=save_pickle('/content/drive/MyDrive/Gab/Variables_saved/T7.pickle',T7)
# f9=save_pickle('/content/drive/MyDrive/Gab/Variables_saved/T9.pickle',T9)


In [58]:
def open_pickle(path_pickle,f):
  f = open(path_pickle, 'rb')
  T = pickle.load(f)
  f.close()

  return T

#call function
# T1=open_pickle('/content/drive/MyDrive/Gab/Variables_saved/T1.pickle',f1)
T3=open_pickle('/content/drive/MyDrive/Gab/Variables_saved/T3.pickle',f3)
T5=open_pickle('/content/drive/MyDrive/Gab/Variables_saved/T5.pickle',f5)
# T7=open_pickle('/content/drive/MyDrive/Gab/Variables_saved/T7.pickle',f7)
# T9=open_pickle('/content/drive/MyDrive/Gab/Variables_saved/T9.pickle',f9)


Final input data and label reshaping

In [73]:

#participant size= number of trials 
participant_size_3= len(T3)
participant_size_5= len(T5)

#X-data with shape (trialxchannelsxframe)=(80trials  x 4channels x (1926frames x 4sec x 2surfaces x n-participants))
T = tf.concat([T3, T5], axis=0) # 2 participants just for testing 
X=T[:, :, 1:5]
print("Data shape:", X.shape)
#Y-categorical encoded label = 1 for asphalt and 0 for grass 
y=T[:,:,6]
print("Label shape:", y.shape)



Data shape: (30816, 80, 4)
Label shape: (30816, 80)


#Basic Convnet

In [ ]:
# Basic Convnet 
from keras import layers 
from keras import models
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1))) model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu')) model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))